In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gudhi as gd

In [ ]:
#LoopForest.py needs to be in same directory
import LoopForest as f

In [ ]:
%load_ext autoreload

%autoreload 3

# Example use case

In [ ]:
rng = np.random.default_rng(34)
num_points=80
points = rng.uniform(low=0.0, high=1.0, size=(num_points,2))

plt.scatter( points[:,0], points[:,1])
plt.show()

In [ ]:
loop_forest = f.compute_loop_forest(points)

## Barcode

The H1 barcode of the point cloud is stored in the LoopForest object loop_forest at loop_forest.barcode as a list of Bar objects. 
Each Bar object bar has attributes bar.birth, bar.death and bar.cycle_reps
The progession of optimal cycle representatives is stored in bar.cycle_reps as list of Loop objects.

Each Loop object has a vertex list which are saves as list of indices in loop.vertex_list. 
The point coordinates can be accessed via loop_forest.point_cloud[vertex] for vertex in loop.vertex_list
Additionally, each Loop has a loop.active_start and loop.active_end attribute which give the interval in which this cycle representative is optimal.

In [ ]:
for bar in loop_forest.barcode:
    print(f"Bar atributes: Birth {bar.birth}, Death {bar.death}, \nand cycle reps {bar.cycle_reps}")

## Plotting functions for Loop Forest Class

In [ ]:
loop_forest.plot_dendrogram()

In [ ]:
#Function also works with only default parameters
loop_forest.plot_barcode(sort="birth")

In [ ]:
for k in range(1,8):
    loop_forest.plot_at_filtration(0.005*k)

In [ ]:
for root in loop_forest.roots:
    print(loop_forest.leaves_below_node(root))